# Get Original Report

In [1]:
# add some features so we can choose from incomestatements,balancesheet,statement of cashflow
reportType = c('is','bs','cf')
link_head = 'http://financials.morningstar.com/ajax/ReportProcess4CSV.html?&t='
link_mid = '&region=usa&culture=en-US&version=SAL&cur=&reportType='
link_tail = '&period=12&dataType=A&order=asc&columnYear=5&curYearPart=1st5year&rounding=3&view=raw&r=769236&denominatorView=raw&number=3'

getfinrpt <- function(cik,rpt){
    if(rpt=='income'){
        links <- paste(link_head,cik,link_mid,reportType[1],link_tail,sep = "")
    } else if (rpt=='balance'){
        links <- paste(link_head,cik,link_mid,reportType[2],link_tail,sep = "")
    } else if (rpt=='cash'){
        links <- paste(link_head,cik,link_mid,reportType[3],link_tail,sep = "")
    }
    data <- read.csv2(links, sep=",",  header=FALSE,blank.lines.skip = FALSE,fill = TRUE,na.strings = "NA",stringsAsFactors = FALSE)
    colnames <- c('category')
    for (i in 2:(ncol(data)))
    {
    colnames[i] <- data[2,i]
    }
    df <- data.frame(data[3:nrow(data),1:ncol(data)],row.names = c(1:(nrow(data)-2)))
    colnames(df) <- colnames
    return(df)
}

In [2]:
#testing
income <- getfinrpt(cik = 'xnas:NFLX',rpt = 'income')
balance <- getfinrpt(cik = 'xnas:NFLX',rpt = 'balance')
cash <- getfinrpt(cik = 'xnas:NFLX',rpt = 'cash')

# DuPont Analysis 

## What is DuPont Analysis
[Accounting Analytics @ Coursera](https://www.coursera.org/learn/accounting-analytics/home)

I will use ratios below to analyze & hopefully make some reasonable forcast.

```r
fin_ratios <- read.csv2('/Users/randall/Downloads/ChromeDownload/ratios.csv', 
                        sep=",",  header=FALSE,blank.lines.skip = FALSE,fill = TRUE,na.strings = "NA",stringsAsFactors = FALSE)

colnames(fin_ratios) <- c('rations','equation')
```

![](http://upload.ouliu.net/i/20180619091012w5zzg.png)

In [3]:
colnames(balance)
colnames(income)
colnames(cash)

[1] "category" "2013-12"  "2014-12"  "2015-12"  "2016-12"  "2017-12"

[1] "category" "2013-12"  "2014-12"  "2015-12"  "2016-12"  "2017-12"  "TTM"

[1] "category" "2013-12"  "2014-12"  "2015-12"  "2016-12"  "2017-12"  "TTM"

In [4]:
#before we started, let's do some consolidation
library(plyr)
csld  <- rbind(balance, income[,1:6],cash[,1:6])

## Return on Equity
- Return on Equity = Net Income / Avg. Stockholders' Equity

In [5]:
NetIncome <- as.numeric(c(csld[50,3:6]))
StockholdersEquity <- as.numeric(csld[35,2:6])
AvgStockholdersEquity <- c()

for (i in 1:4){
    AvgStockholdersEquity[i] <- 0.5*(StockholdersEquity[i]+StockholdersEquity[i+1])
}

ROE <- NetIncome/AvgStockholdersEquity

### Financial Leverage

- Return on Assets = De-levered Net Income / Avg. Total Assets

In [7]:
TotalAssets <- as.numeric(csld[16,2:6])
AvgTotalAssets<- c()

for (i in 1:4){
    AvgTotalAssets[i] <- 0.5*(TotalAssets[i]+TotalAssets[i+1])
}

ROA <- NetIncome/AvgTotalAssets

- Financial Leverage = Avg. Total Assets / Avg. Stockholders’ Equity

In [8]:
FinancialLeverage <- AvgTotalAssets/AvgStockholdersEquity

As for Statutory Tax Rate, I will assume 38% for now

- Correction Factor = Net Income / De-levered Net Income
- De-levered Net Income = Net Income + (Interest Exp. * (1 - Statutory Tax Rate))

[Effective Tax Rates Can Differ Significantly from the Statutory Rate](https://www.gao.gov/products/GAO-13-520)

In [9]:
StatutoryTaxRate  <- 0.38
InterestExp <- as.numeric(csld[45,3:6])
DeleveredNetIncome <-  NetIncome + (InterestExp*(1-StatutoryTaxRate))

CorrectionFactor <- NetIncome/DeleveredNetIncome

### Return on Assets
- Return on Sales =  De-levered Net Income / Sales 
- Asset turnover = Sales / Avg. Total Assets 

In [10]:
Sales <- as.numeric(csld[37,3:6])
ROS <- DeleveredNetIncome / Sales 
AssetTurnover <- Sales / AvgTotalAssets

In [11]:
ratios <- data.frame(ROE, ROA,FinancialLeverage,ROS,AssetTurnover,row.names = c('FY14','FY15','FY16','FY17'))

In [12]:
t(ratios)
#https://cran.r-project.org/web/packages/formattable/vignettes/formattable-data-frame.html

,FY14,FY15,FY16,FY17
ROE,0.16729323,0.06027934,0.07627983,0.17853721
ROA,0.04282277,0.01425261,0.01572089,0.03429448
FinancialLeverage,3.90664160,4.22935555,4.85213135,5.20600447
ROS,0.05447048,0.03313864,0.03170649,0.06042590
AssetTurnover,0.88291901,0.78563152,0.74241278,0.71736196


#### Return on Sales
##### Profitability

- Gross Margin = (Sales - Cost of Goods Sold) / Sales 
- SGA as of Sales = SGA Expense / Sales 
- Operating Margin = Operating Income / Sales 
- Interest Expense as  of Sales = Interest Expense / Sales 
- Effective Tax Rate = Income Taxes / Pre-tax Income 

[How to Calculate the Effective Tax Rate of a Corporation](https://www.fool.com/knowledge-center/how-to-calculate-the-effective-tax-rate-of-a-corpo.aspx)

[SEC FILINGS](https://www.sec.gov/Archives/edgar/data/1065280/000106528018000069/q4nflx201710k.htm#sE1D5BA7DC75D59B9843BA1F37B4C71EA)


In [13]:
GrossMargin  <- (Sales-as.numeric(csld[39,3:6]) / Sales)

In [14]:
SGAasofSales <- as.numeric(csld[42,3:6]) / Sales

In [15]:
OperatingMargin <- as.numeric(csld[44,3:6]) / Sales

In [16]:
InterestExpenseasofSales <- as.numeric(csld[45,3:6]) / Sales

In [17]:
InterestExpenseasofSales <- as.numeric(csld[48,3:6]) / as.numeric(csld[47,3:6]) 

##### Asset Turnover Ratios		
> NETFLIX receivables from members generally settle quickly and deferred revenue is a source of cash flow.

- Accounts Receivables Turnover = Sales / Avg. Accounts Receivable
- Inventory Turnover = Cost of Goods Sold / Avg. Inventory
- Accounts Payable Turnover = Purchases / Avg. Accounts Payable
- Fixed Asset Turnover = Sales / Avg. Net Property, Plant and Equipment

- Accounts Receivables Turnover = Sales / Avg. Accounts Receivable
- Accounts Payable Turnover = Purchases / Avg. Accounts Payable

##### Days Turnover Ratios		
- Days Receivables =  365 * (Avg. Accounts Receivable / Sales)
- Days Inventory = 365 * (Avg. Inventory / Cost of Goods Sold)
- Days Payable = 365 * (Avg. Accounts Payable / Purchases)
- Net Trade Cycle = Days Receivable + Days Inventory - Days Payable
- Net Trade Cycle = Days Receivable + Days Inventory

#### Liquidity Analysis		
- Current Ratio = Current Assets / Current Liabilities
- Quick Ratio = (Cash + Accts Rec) / Current Liabilities
- CFO-to-Current Liabilities = Cash from Operations / Avg. Current Laibilities
		
- Interest Coverage = Operating Income before Depreciation / Interest Expense
- Cash Interest Coverage = (Cash from Operations + Cash Interest + Cash Taxes) / Cash Interest Paid
		
- Debt to Equity = Total Liabilities / Total Stockholders’ Equity
- Long-Term-Debt to Equity = Total Long-Term Debt / Total Stockholders’ Equity
- Long-Term Debt to Tangible Assets = Total Long-Term Debt / (Total Assets - Intangible Assets)

In [18]:
CurrentRatio  <- as.numeric(csld[8,3:6]) / as.numeric(csld[19,3:6])

In [19]:
CurrentLaibilities <- as.numeric(csld[24,2:6])
AvgCurrentLaibilities <- c()

for (i in 1:4){
    AvgCurrentLaibilities[i] <- 0.5*(CurrentLaibilities[i]+CurrentLaibilities[i+1])
}

CFOtoCurrentLiabilities <- as.numeric(csld[70,3:6]) / AvgCurrentLaibilities

## Other Information		
- Sales Growth = Change in Sales / Prior Year Sales
- Federal Statutory Rate + State and Foreign Statutory Rates (excludes permanent differences)
- De-levered Net Income = Net Income + (Interest Exp. * (1 - Statutory Tax Rate))
- Purchases = Ending Inventory + COGS – Beginning Inventory
- Weighted Avg Depreciation Rate = Depreciation  Amortization / (Avg. Gross PPE + Avg. Intangible Assets)
- Weighted Avg Interest Rate = Interest Expense / Avg. Long-Term Debt